In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Disables INFO & WARNING logs
import tensorflow as tf
import keras
import numpy as np
from utils import center_crop
import matplotlib.pyplot as plt
from maptd_model import maptd_model

In [2]:
model = maptd_model(input_shape=(512, 512, 3))

ckpt_dir = './data/ckpts/models/0/gan_ckpts'
latest = tf.train.latest_checkpoint(ckpt_dir)
print(latest)
ckpt_prefix = os.path.join(ckpt_dir, 'ckpt')
ckpt = tf.train.Checkpoint(model=model)
ckpt.restore(latest)


94765736/94765736 [==============================] - 1s 0us/step
./data/ckpts/models/0/gan_ckpts/ckpt-50


In [4]:
modelpath = './data/ckpts/models/0/gan_modified'

In [5]:
model = keras.models.load_model(modelpath)
#model = maptd_model(input_size=512)

OSError: SavedModel file does not exist at: ./data/ckpts/models/0/gan_modified\{saved_model.pbtxt|saved_model.pb}

In [5]:
model_config = model.get_config()

In [6]:
input_layer_name = model_config['layers'][0]['name']

In [7]:
model_config['layers'][0]
model_config['layers'][0] = {
                      'name': 'new_input',
                      'class_name': 'InputLayer',
                      'config': {
                          'batch_input_shape': (None, None, None, 3),
                          'dtype': 'float32',
                          'sparse': False,
                          'name': 'new_input'
                      },
                      'inbound_nodes': []
                  }

model_config['layers'][1]['inbound_nodes'] = [[['new_input', 0, 0, {}]]]
model_config['input_layers'] = [['new_input', 0, 0]]
model_config['layers'][-5]['config']['activation'] = tf.atan # NOTE: I added this modification, because I forgot to add the tf.atan as a custom object of keras when I created the model
new_model = model.__class__.from_config(model_config, custom_objects={})  # change custom objects if necessary

In [7]:
# iterate over all the layers that we want to get weights from
weights = [layer.get_weights() for layer in model.layers[1:]]
for layer, weight in zip(new_model.layers[1:], weights):
    layer.set_weights(weight)

In [10]:
new_model.save(modelpath)

INFO:tensorflow:Assets written to: ./data/ckpts/models/0/gan_modified/assets


INFO:tensorflow:Assets written to: ./data/ckpts/models/0/gan_modified/assets


In [ ]:
img = np.expand_dims(plt.imread('D:/Gerasimos/Toponym_Recognition/MapTD_General/MapTD_TF2/data/general_dataset/images/D5005-5028149.tiff'), axis=0)
img = img[:, :6144, :7168, :]
img.shape
#img = center_crop(img, crop_shape=(2560, 2560))
p = model(img, training=False)

In [ ]:
plt.imshow(np.where(p[0][0] > 0.8, 1, 0), cmap='gray')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, dpi=500)
ax[0].imshow(np.where(p[0][0] > 0.8, 1, 0), cmap='gray')
plt.show()